In [1]:
%cd ..

d:\~Perso\Etudes\DAC\NeuralNetworksDIY


In [2]:
import numpy as np
from src.convolution import *
from utils.mltools import *


def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy


alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


In [3]:
import os
import pandas as pd
from skimage import io, transform
from PIL import Image


def load_data(root_dir):
    size = (540, 420)
    X = []
    for filepath in os.listdir(root_dir + "/X"):
        imgpath = os.path.join(root_dir, "X", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        X.append(image)
    X = np.array(X)

    y = []
    for filepath in os.listdir(root_dir + "/y"):
        imgpath = os.path.join(root_dir, "y", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        y.append(image)
    y = np.array(y)

    return X, y


X, y = load_data("./data/dirty_documents")


In [4]:
X[:, np.newaxis, :, :].shape


(144, 1, 540, 420)

In [5]:
X[:, :, :, np.newaxis].shape


(144, 540, 420, 1)

## Vectorisation


In [14]:
input = np.arange(25).reshape(5, 5)

kernel_size = 4
layer_stride = 1

height, width = input.shape
rows_stride, columns_strides = input.strides

out_height = int((height - kernel_size) / layer_stride + 1)
out_width = int((width - kernel_size) / layer_stride + 1)

new_shape = (out_height, out_width, kernel_size, kernel_size)
new_strides = (
    rows_stride * layer_stride,
    columns_strides * layer_stride,
    rows_stride,
    columns_strides,
)

windowed_input = np.lib.stride_tricks.as_strided(input, new_shape, new_strides)
print(
    windowed_input,
    f"\nShape: {windowed_input.shape}, \tStrides: {windowed_input.strides}",
)


[[[[ 0  1  2  3]
   [ 5  6  7  8]
   [10 11 12 13]
   [15 16 17 18]]

  [[ 1  2  3  4]
   [ 6  7  8  9]
   [11 12 13 14]
   [16 17 18 19]]]


 [[[ 5  6  7  8]
   [10 11 12 13]
   [15 16 17 18]
   [20 21 22 23]]

  [[ 6  7  8  9]
   [11 12 13 14]
   [16 17 18 19]
   [21 22 23 24]]]] 
Shape: (2, 2, 4, 4), 	Strides: (20, 4, 20, 4)


In [15]:
input = np.arange(5)

kernel_size = 2
layer_stride = 3

length = input.shape[0]
rows_stride = input.strides[0]

out_height = int((length - kernel_size) / layer_stride + 1)

new_shape = (
    out_height,
    kernel_size,
)
new_strides = (
    rows_stride * layer_stride,
    rows_stride,
)

windowed_input = np.lib.stride_tricks.as_strided(input, new_shape, new_strides)
print(
    windowed_input,
    f"\nShape: {windowed_input.shape}, \tStrides: {windowed_input.strides}",
)


[[0 1]
 [3 4]] 
Shape: (2, 2), 	Strides: (12, 4)


## Vérif Conv


In [29]:
import torch
import torch.nn as nn

convtest = nn.Conv2d(1, 3, 4, stride=1, bias=False)
A = torch.from_numpy(X[:, np.newaxis, :, :])
convtest(A.float()).shape


In [ ]:
c2d = Conv2D(1, 3, 2)
out = c2d.forward(X[:, np.newaxis, :, :])
out.shape
